In [5]:
import time
import os
import subprocess
import cv2
import moviepy
from skimage.metrics import structural_similarity as ssim
from PIL import Image
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [14]:
def capture_video(folder_name, delay_s=3, video_duration_s = 5):
    folder_path = f"../data/videos/{folder_name}"
    os.makedirs(folder_path, exist_ok=True)

    print("\nTaking video in...")
    while delay_s > 0:
        print(f"{delay_s}...")
        time.sleep(1)
        delay_s -= 1

    raw_video_path = f"{folder_path}/i420.avi"
    command = f"ffmpeg -y -f v4l2 -framerate 30 -i /dev/video0 -t {video_duration_s} {raw_video_path}"
    subprocess.run(command, shell=True)

    formats = {
        "mp4.mp4": "libx264",   # MP4 format
        "divx.avi": "mpeg4",    # DivX format
        "flv.flv": "flv",       # FLV format
        "xvid.avi": "libxvid"   # Xvid format
    }

    for filename, codec in formats.items():
        output_path = f"{folder_path}/{filename}"
        command = f"ffmpeg -i {raw_video_path} -c:v {codec} {output_path}"
        subprocess.run(command, shell=True)


In [ ]:
capture_video("test")


Taking video in...
3...
